In [1]:
import os
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))

In [3]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [59]:
phase_num = 3
step_optim = 1
step_forecast = 5
b_num = 1
df = pd.read_csv(f'debug_logs/pow_gurobi_phase_{phase_num}_step_leap_{step_optim}_forecast_step_{step_forecast}.csv', index_col='time_step')
tog_forecast_temp = df[df['building'] == b_num]
df

,building,+0h,+1h,+2h,+3h,+4h,+5h,+6h,+7h,+8h,...,+14h,+15h,+16h,+17h,+18h,+19h,+20h,+21h,+22h,+23h
time_step,,,,,,,,,,,,,,,,,,,,,
0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.470093,...,0.691382,-0.266271,-0.711764,-1.231934,-1.512884,-1.355655,0.000000,-0.752148,0.000000,0.000000
0,1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.445560,...,0.669968,-0.280011,-0.681627,-1.181309,-1.433856,-1.314948,-0.404932,0.000000,-0.533972,0.000000
0,2,0.0,0.000000,0.000000,0.000000,0.041084,-0.034099,0.0,0.000000,0.472431,...,0.691382,-0.266105,-0.711912,-1.231934,-1.512884,-1.354668,0.000000,-0.753153,0.000000,0.000000
0,3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.504853,...,0.674543,-0.260035,-0.671301,-1.186248,-1.439446,-1.337567,-0.918601,0.000000,0.000000,-0.017460
0,4,0.0,0.000000,0.024800,0.089402,0.000000,-0.094787,0.0,0.000000,0.445560,...,0.669968,-0.280011,-0.681627,-1.181309,-1.433856,-1.314948,0.000000,-0.498140,-0.440764,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8759,2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,1.598829,0.916236,0.000000,0.000000,0.000000,-0.793366,-0.773687,-1.063134,-0.944060,-2.175488
8759,3,0.0,1.025577,1.503108,0.000000,0.000000,0.000000,0.0,1.290961,2.297491,...,0.000000,0.000000,0.000000,0.000000,-1.354506,-1.412085,-1.500392,-1.025895,0.000000,-0.537777
8759,4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,-0.865566,-1.819645,-1.252357,-0.301022,-0.345424,-0.290214,-0.276149,-0.250423


In [60]:
df_mod = pd.DataFrame()
for b_num in df['building'].unique():
    tog_forecast_temp = df[df['building'] == b_num]
    # impute missing time_steps in tog_forecast_temp with shifter versions of the previous row
    complete_index = pd.DataFrame(index=range(tog_forecast_temp.index.min(), tog_forecast_temp.index.max() + 1))
    # Merge the new DataFrame with the original DataFrame and forward fill missing values
    merged_df = complete_index.merge(tog_forecast_temp, left_index=True, right_index=True, how='left')
    #merged_df['time_step'] = merged_df['time_step'].fillna(method='ffill')
    #merged_df['building'] = merged_df['building'].fillna(method='ffill')
    for col in merged_df.columns:
        merged_df[col] = merged_df[col].fillna(method='ffill')
    # for cols in merged_df.columns[1:] shift values to the right by 1
    merged_df['time_step'] = merged_df.index
    build_col = merged_df['building']
    merged_df_tp = merged_df.T.drop(index=['time_step', 'building'])
    # shift each column by 1 in first five columns
    for i in range(len(merged_df_tp.columns) // step_optim):
        for j, col in enumerate(merged_df_tp.columns[step_optim*i:step_optim*(i+1)]):
            merged_df_tp[col] = merged_df_tp[col].shift(-j)
    merged_df = merged_df_tp.T
    merged_df['building'] = build_col.astype(int)
    # name index time_step
    merged_df.index.name = 'time_step'
    # birng the building column to the front
    merged_df = merged_df[['building'] + [col for col in merged_df.columns if col != 'building']]
    # append to df_mod
    df_mod = pd.concat([df_mod, merged_df], axis=0)
df_mod

,building,+0h,+1h,+2h,+3h,+4h,+5h,+6h,+7h,+8h,...,+14h,+15h,+16h,+17h,+18h,+19h,+20h,+21h,+22h,+23h
time_step,,,,,,,,,,,,,,,,,,,,,
0,0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.470093,...,0.691382,-0.266271,-0.711764,-1.231934,-1.512884,-1.355655,0.000000,-0.752148,0.000000,0.000000
1,0,0.000000,0.032786,0.000000,0.04907,-0.067940,0.000000,0.000000,0.470093,0.876611,...,-0.266271,-0.711764,-1.231934,-1.512884,-1.355655,0.000000,-0.752148,0.000000,0.000000,0.000000
2,0,0.000000,0.097388,0.049070,-0.12156,0.000000,0.000000,0.470093,0.876611,1.139393,...,-0.711764,-1.231934,-1.512884,-1.355655,-0.752148,0.000000,0.000000,0.000000,0.000000,0.000000
3,0,0.000000,0.049070,-0.040728,0.00000,0.000000,0.470093,0.876611,1.139393,1.261346,...,-1.231934,-1.512884,-1.355655,0.000000,0.000000,-0.752148,0.000000,0.000000,0.000000,0.000000
4,0,0.000000,0.000000,0.000000,0.00000,0.470093,0.876611,1.139393,1.261346,1.435737,...,-1.512884,-1.355655,-0.752148,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,6,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.260428,0.289182,0.133497,0.000000,0.000000,0.000000,-0.857397,-2.102207,-2.871051,0.000000
8756,6,-0.110616,-2.495384,-0.100919,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.289182,2.913872,0.709565,0.000000,0.000000,0.000000,-1.674836,-2.125520,0.000000,-2.030300
8757,6,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.466534,0.000000,...,0.133497,0.000000,0.000000,0.000000,0.000000,0.000000,-0.473446,-3.419108,-1.542559,-0.395543


In [63]:
df_mod.sort_values(by=['time_step', 'building'], inplace=True)
df_mod

,building,+0h,+1h,+2h,+3h,+4h,+5h,+6h,+7h,+8h,...,+14h,+15h,+16h,+17h,+18h,+19h,+20h,+21h,+22h,+23h
time_step,,,,,,,,,,,,,,,,,,,,,
0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.470093,...,0.691382,-0.266271,-0.711764,-1.231934,-1.512884,-1.355655,0.000000,-0.752148,0.000000,0.000000
0,1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.445560,...,0.669968,-0.280011,-0.681627,-1.181309,-1.433856,-1.314948,-0.404932,0.000000,-0.533972,0.000000
0,2,0.0,0.000000,0.000000,0.000000,0.041084,-0.034099,0.0,0.000000,0.472431,...,0.691382,-0.266105,-0.711912,-1.231934,-1.512884,-1.354668,0.000000,-0.753153,0.000000,0.000000
0,3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.504853,...,0.674543,-0.260035,-0.671301,-1.186248,-1.439446,-1.337567,-0.918601,0.000000,0.000000,-0.017460
0,4,0.0,0.000000,0.024800,0.089402,0.000000,-0.094787,0.0,0.000000,0.445560,...,0.669968,-0.280011,-0.681627,-1.181309,-1.433856,-1.314948,0.000000,-0.498140,-0.440764,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8759,2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,1.598829,0.916236,0.000000,0.000000,0.000000,-0.793366,-0.773687,-1.063134,-0.944060,-2.175488
8759,3,0.0,1.025577,1.503108,0.000000,0.000000,0.000000,0.0,1.290961,2.297491,...,0.000000,0.000000,0.000000,0.000000,-1.354506,-1.412085,-1.500392,-1.025895,0.000000,-0.537777
8759,4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,-0.865566,-1.819645,-1.252357,-0.301022,-0.345424,-0.290214,-0.276149,-0.250423


In [58]:
for step_optim in tqdm(range(2, 25)):
    for steps_skip_forecast in range(step_optim, 25):
        df = pd.read_csv(f'debug_logs/pow_gurobi_phase_{phase_num}_step_leap_{step_optim}_forecast_step_{steps_skip_forecast}.csv', index_col='time_step')
        df_mod = pd.DataFrame()
        for b_num in df['building'].unique():
            tog_forecast_temp = df[df['building'] == b_num]
            # impute missing time_steps in tog_forecast_temp with shifter versions of the previous row
            complete_index = pd.DataFrame(index=range(tog_forecast_temp.index.min(), tog_forecast_temp.index.max() + 1))
            # Merge the new DataFrame with the original DataFrame and forward fill missing values
            merged_df = complete_index.merge(tog_forecast_temp, left_index=True, right_index=True, how='left')
            #merged_df['time_step'] = merged_df['time_step'].fillna(method='ffill')
            #merged_df['building'] = merged_df['building'].fillna(method='ffill')
            for col in merged_df.columns:
                merged_df[col] = merged_df[col].fillna(method='ffill')
            # for cols in merged_df.columns[1:] shift values to the right by 1
            merged_df['time_step'] = merged_df.index
            build_col = merged_df['building']
            merged_df_tp = merged_df.T.drop(index=['time_step', 'building'])
            # shift each column by 1 in first five columns
            for i in range(len(merged_df_tp.columns) // step_optim):
                for j, col in enumerate(merged_df_tp.columns[step_optim*i:step_optim*(i+1)]):
                    merged_df_tp[col] = merged_df_tp[col].shift(-j)
            merged_df = merged_df_tp.T
            merged_df['building'] = build_col.astype(int)
            # name index time_step
            merged_df.index.name = 'time_step'
            # birng the building column to the front
            merged_df = merged_df[['building'] + [col for col in merged_df.columns if col != 'building']]
            # append to df_mod
            df_mod = pd.concat([df_mod, merged_df], axis=0)
        df_mod.to_csv(f'debug_logs/pow_gurobi_phase_{phase_num}_step_leap_{step_optim}_forecast_step_{steps_skip_forecast}_modifed.csv')


100%|██████████| 23/23 [1:00:33<00:00, 157.97s/it]


In [64]:
for step_optim in tqdm(range(2, 25)):
    for steps_skip_forecast in range(step_optim, 25):
        df_mod = pd.read_csv(f'debug_logs/pow_gurobi_phase_{phase_num}_step_leap_{step_optim}_forecast_step_{steps_skip_forecast}_modifed.csv', index_col=0)
        df_mod.sort_values(by=['time_step', 'building'], inplace=True)
        df_mod.to_csv(f'debug_logs/pow_gurobi_phase_{phase_num}_step_leap_{step_optim}_forecast_step_{steps_skip_forecast}_modifed.csv')



100%|██████████| 23/23 [03:50<00:00, 10.04s/it]
